In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import SGD, Adam
from keras.utils.np_utils import to_categorical
from keras import backend as K
from keras.constraints import maxnorm
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(y_train.shape)
print(type(y_train))

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255.0
x_test = x_test/255.0
y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)
num_classes = y_test.shape[1]
print(num_classes)
print('x_train_shape: ', x_train.shape)
print('test_shape: ', y_train.shape)


In [ ]:
print(x_train[:1])

In [ ]:
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_train[i]) 
plt.show()

In [ ]:
print("x_train_shape: ", x_train.shape)
print("y_train_shape: ", y_train.shape)
print("x_test_shape: ", x_test.shape)
print("y_test_shape: ", y_test.shape)


In [ ]:
optimizer = Adam(learning_rate=0.001)
epochs = 50
batch_size = 100

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32,kernel_size = (5,5), padding='same',
                 activation = 'relu', input_shape = (32,32,3)))
model.add(Conv2D(filters = 32,kernel_size = (3,3), padding='same',
                 activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 64,kernel_size = (3,3), padding='same',
                 activation = 'relu'))
model.add(Conv2D(filters = 32,kernel_size = (3,3), padding='same',
                 activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
datagen = ImageDataGenerator(
            rotation_range = 0.5,
            width_shift_range = 0.5,
            height_shift_range = 0.5,
            zoom_range = 0.5)
datagen.fit(x_train)

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs = epochs,
                              validation_data=(x_test, y_test), steps_per_epoch = batch_size)

In [ ]:
results = model.evaluate(x_test, y_test)

In [ ]:
print(x_test[0].shape)

In [ ]:
model.save_weights('model_28_06_2022_20_26.h5')

In [ ]:
import cv2


img = cv2.imread('../input/kamyonet-resim/kamyonet.jpg')
img2 = cv2.imread('../input/birdimage/cat.jpg')
img, img2 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB),cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)
img, img2 = cv2.resize(img,(32,32), interpolation = cv2.INTER_AREA), cv2.resize(img2,(32,32), interpolation = cv2.INTER_AREA)
img,img2 = np.array(img),np.array(img2)
print(img.shape)
print(img2.shape)
plt.subplot(221)
plt.imshow(img)
plt.subplot(222)
plt.imshow(img2)
plt.show()


In [ ]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
img_array = np.expand_dims(img2, axis=0)
prediction  = model.predict(img_array)
print(prediction[0])
prediction = np.around(prediction)
x = prediction[0].astype(int)
print(prediction[0])
print(class_names[np.where(x == 1)[0][0]])

In [ ]:
plt.plot(history.history['loss'])
plt.show()